# Lecture 9b

## Function approximation by the finite element method

We have seen that a function $u(x)$ can be approximated as

$$
u(x) = \sum_{j=0}^N \hat{u}_j \psi_j(x), \quad x \in \Omega.
$$

And we have seen how the least squares, Galerkin or collocation methods can be used to find the unknown $\boldsymbol{\hat{u}} = \{\hat{u}_j\}_{j=0}^N$.

Up until now we have used global basis functions $\psi_j(x)$ defined on the entire domain $\Omega$. For example, the Chebyshev polynomials are all functions that are changing within the entire domain $\Omega = [-1, 1]$

In [ ]:
import numpy as np
import sympy as sp 
import matplotlib.pyplot as plt 

x = sp.Symbol('x')
fig, ax = plt.subplots(figsize=(6, 3))
xj = np.linspace(-1, 1, 100)
legend = []
for n in range(5):
    ax.plot(xj, np.cos(n*np.arccos(xj)))
    legend.append(f'$P_{n}(x)$')
ax.legend(legend);

The global aspect of the basis functions is an advantage when it comes to both accuracy and efficiency. However, we are not always interested in solving equations on a simple line interval, or a rectangle for two dimensions.

Normally you are more interested in domains that contains some physical obstructions

```{figure} dolfin_mesh.png
:name: dolfin mesh
:width: 300
:align: left
```

For such domains it is not possible to use basis functions that are defined everywhere. It is also very difficult, if not impossible, to use finite difference methods. Just imagine, how would you implement a Laplacian, like

$$
\frac{u_{i+1,j}-2u_{i,j}+u_{i-1,j}}{\Delta x^2} + \frac{u_{i,j+1}-2u_{i,j}+u_{i,j-1}}{\Delta y^2} 
$$

on a 2D dolfin mesh?

But finite element methods are designed to work on such triangulated elements! 

The finite element method makes use of *local* basis functions that are only non-zero on a small part of the total domain. Furthermore, the basis functions that we will make use of in this class are piecewise polynomials. In the simplest possible form, this means that the basis functions are piecewise linear. For example, if the domain is the part of the real line $\Omega = [0, 5]$ and we make use of a uniform discretization with 5 intervals, then we get 5 elements as shown $\Omega^{(e)}$, and 6 piecewise linear basis functions. Below we plot only two of the 6 basis functions for simplicity.

```{figure} fe_mesh1D_phi_2_3.png
:name: fem-1d-mesh
:width: 500
:align: left
```

### Finite element basis functions

Other than the fact that the basis functions are piecewise polynomials with only local support, there is nothing new from the previous chapters. It is only much more complicated to work with piecewise polynomials than continuous and it requires much more effort to implement.

The finite element method is a variational method. As such we can define the problem of approximating functions exactly the same way as we did for the global approach. Assume that we want to use the finite element method to find an approximation to $u(x)$ in $\Omega=[0, 5]$. We may then divide this interval into $5$ nonoverlapping elements (like shown above) and use the function space $V_N = \text{span}\{\psi_j\}_{j=0}^5$, where $\psi_j(x)$ are the 6 piecewise linear functions that are 1 on node $j$, zero on all the other nodes and linear in between. This would be the 6 basis functions that all would look like the 2 we have plotted above. We then attempt to find $u_N \in V_N$ such that

$$
(u-u_N, v) = 0 \quad \forall \, v \in V_N.
$$

It is also possible to use the least squares method, but this is very rare.

So the finite element method we will focus on is the Galerkin method with piecewise polynomial basis functions.

We will reuse even more of the theory from the global approximation, because we will make use of Lagrange polynomials as basis functions! But not global Lagrange polynomials, only local this time. Local to an element. Each element in the {ref}`1D fem mesh <fem-1D-mesh>` contains $N_e = 2$ nodes. These nodes will locally be denoted as $(x^0, x^1)$ no matter which element we are in, and the Lagrange formula is then

$$
\ell_j(x) = \prod_{\substack{0 \le m < 2 \\ m \ne j}} \frac{x-x^m}{x^j-x^m}, \quad j \in (0, 1).
$$

There are exactly two Lagrange polynomials on each element, and these are locally computed as

$$
\ell_0(x) = \frac{x-x^1}{x^0-x^1} \quad \text{and} \quad \ell_1(x) = \frac{x-x^0}{x^1-x^0}.
$$

Since the mesh is uniform $x^{1}-x^0 = 1$. As such,

$$
\ell_0(x) = x^1-x \quad \text{and} \quad \ell_1(x) = x-x^0.
$$

Element 0 contains mesh points $(x^0, x^1) = (x_0, x_1) = (0, 1)$. We get

$$
\psi_0(x) = \ell_0(x) = 1-x \quad \text{and} \quad \psi_1(x)= \ell_1(x) = x \quad x \in [0, 1].
$$

where $\psi_j(x)$ is the basis function for node $j$. It is one on node j and 0 on all the remaining nodes in the mesh.

Element 1 contains mesh points $(x^0, x^1) = (x_1, x_2) = (1, 2)$. We get

$$
\psi_1(x) = \ell_0(x) = 2-x \quad \text{and} \quad \psi_2(x) = \ell_1(x) = x-1 \quad x \in [1, 2].
$$

Note that we get $\psi_1(x)$ in both element 0 and 1. This is because the basis function is piecewise linear, with one slope on element 0 and another on element 1.

It should be obvious that element $j$ for $j=2, 3, 4$ contains basis functions

$$
\psi_j(x) = j+1-x \quad \text{and} \quad \psi_{j+1}(x) = x-j, \quad x \in [j, j+1]
$$

We can plot all these basis functions with one color for each basis function $\psi_j(x)$

In [ ]:
color = iter(plt.cm.rainbow(np.linspace(0, 1, 7)))
c = next(color)
plt.figure(figsize=(6, 3))
for j in range(5):
    x = np.array([j, j+1])
    plt.plot(x,  j+1-x, c=c)
    c = next(color)
    plt.plot(x,  x-j, c=c)

So all the internal basis functions, i.e., $\{\psi_j(x)\}_{j=1}^{4}$, are nonzero in two elements each and zero elsewhere. The two boundary basis functions $\psi_0(x)$ and $\psi_5(x)$ are only nonzero on one element each. All basis functions are such that

$$
\psi_j(x_i) = \delta_{ij} = \begin{cases}
1 \quad i=j \\
0 \quad i\ne j
\end{cases}
$$

since we are using Lagrange polynomials. This applies not only for uniform meshes, but for any collection of mesh points. Assume that the mesh points use Chebyshev points instead. This is no problem for the elements of the Lagrange polynomials.

In [ ]:
xj = np.cos(np.arange(6)*np.pi/5)[::-1]
color = iter(plt.cm.rainbow(np.linspace(0, 1, 7)))
c = next(color)
plt.figure(figsize=(6, 3))
for j in range(5):
    x = np.array([xj[j], xj[j+1]])
    plt.plot(x,  (x-x[1])/(x[0]-x[1]), c=c)
    c = next(color)
    plt.plot(x,  (x-x[0])/(x[1]-x[0]), c=c)

The flexibility when it comes to mesh is one of the major advantages of the finite element method.

For any mesh we get the piecewise linear Lagrange basis functions

$$
\psi_j(x) = \begin{cases}
\frac{x-x_{j-1}}{x_{j}-x_{j-1}} \quad x \in [x_{j-1}, x_{j}],\\
\frac{x-x_{j+1}}{x_{j}-x_{j+1}} \quad x \in [x_{j}, x_{j+1}]. \\
\end{cases}
$$

Lets use these basis functions for the Galerkin method in order to approximate $u(x) = 10(x-1)^2-1, x \in [1, 2]$. We have the function space $V_N = \text{span}\{\psi_j\}_{j=0}^N$ and look for $u_N \in V_N$ such that

$$
(u-\sum_{j=0}^N \hat{u}_j \psi_j, \psi_i) = 0 \quad \forall \, i \in (0, 1, \ldots, N).
$$

In other words, we need to solve 

$$
\sum_{j=0}^N(\psi_j, \psi_i) \hat{u}_j = (u, \psi_i), \quad \forall\,  i \in (0, 1, \ldots, N)
$$

Unfortunately, the mass matrix $(\psi_j, \psi_i)$ is not diagonal, because the basis functions are not orthogonal. However, since the basis functions are local the matrix will still be highly sparse.

Lets create a uniform mesh and assemble the mass matrix. We assemble element by element and we know that on each element there are only two nonzero basis functions

$$
\ell_0(x) = \frac{x-x^1}{x^0-x^1} \quad \text{and} \quad \ell_1(x) = \frac{x-x^0}{x^1-x^0}.
$$

In [ ]:
import sympy as sp
x = sp.Symbol('x')
N = 8
xj = np.linspace(1, 2, N+1)
l0 = lambda j: (x-xj[j+1])/(xj[j]-xj[j+1])
l1 = lambda j: (x-xj[j])/(xj[j+1]-xj[j])
A = np.zeros((N+1, N+1))
for i in range(N):
    A[i, i] += sp.integrate(l0(i)**2, (x, xj[i], xj[i+1]))
    A[i+1, i+1] += sp.integrate(l1(i)**2, (x, xj[i], xj[i+1]))
    aij = sp.integrate(l0(i)*l1(i), (x, xj[i], xj[i+1]))
    A[i, i+1] += aij
    A[i+1, i] += aij

In [ ]:
A